## 101 -- Debug and Monitor Your Anyscale Application

### Introduction

The Anyscale platform provides several tools to monitor and troubleshoot your applications. In this section we’ll cover:

- **Anyscale Log Viewer Tab:** provides persistent access for developers to query, view and download log files generated from your Ray apps, even after cluster termination.
- **Anyscale Metrics Tab:** Anyscale provides built-in metrics for monitoring your Ray clusters and the workloads you run on them
- **Ray Dashboard Tab:** Ray provides a web-based dashboard for monitoring and debugging Ray applications.

Open your existing **Anyscale Workspace** (or spin up a new one). Modify the node group configuration to include lightweight CPU nodes (e.g. m5 on Amazon EC2). Enable autoscaling to scale out from 0 to 10.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging1.png"  width="400"/>


Create a Python file named `estimate_pi.py` and paste the following code. **Run the workflow.** 

This script uses Ray Core to estimate π by breaking the problem into many small tasks (100 submitted jobs) that are distributed across a cluster. Each task does a chunk of the work and some may print a quick update, so you can track the progression in the logs. Finally, it gathers all the results and prints the final estimate of π.

In [ ]:
import ray
import random
import time

@ray.remote
def pi4_sample(sample_count, task_id):
    in_count = 0
    for _ in range(sample_count):
        x = random.random()
        y = random.random()
        if x * x + y * y <= 1:
            in_count += 1

    if task_id % 10 == 0:
        print(f"[Task {task_id}] done")  # random flag for 10% of completed tasks
    return in_count / sample_count

@ray.remote
def log_result(pi, duration):
    print(f"Pi: {pi:.6f}, duration: {duration:.2f}s") # can also use logs instead of prints 

if __name__ == "__main__":
    ray.init() 

    # number of tasks submitted
    batches = 100 
    sample_count = 1000

    print(f"Submitting {batches} tasks...") 
    start = time.time()

    results = [pi4_sample.remote(sample_count, i) for i in range(batches)]
    output = ray.get(results)

    pi = sum(output) * 4 / len(output)
    duration = time.time() - start

    # Log final result inside a Ray object
    ray.get(log_result.remote(pi, duration))

### Exploring the **Anyscale Log Viewer**

While the Ray Application is being processed by the cluster, head over to the **Logs Tab.** Check out the **Application Logs** first. 

Application logs show logs from the code that you execute on your cluster. Logs are categorized by component, and you can use filters or the search box to find specific logs.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging2.png"  width="400"/>

You can parse logs using string matching, regular expressions, and filters such as time range and other metadata.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging3.png"  width="700"/>

Now, check out the Workspace events. This view tracks the logs related to the cluster’s hardware, including when the cluster comes online or goes offline, scaling events, and spot instance preemptions. These logs are retained even if the cluster is offline.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging5.png"  width="700"/>



Logs are saved even after the cluster ends. You can access logs from completed Anyscale Job or Service clusters using the Anyscale CLI. To download logs after the cluster has terminated, run the CLI command below.

You can find the cluster ID locked in one of the environment variables: https://docs.anyscale.com/resources/environment-variables/

```
# Change `cluster_id` to your cluster ID
anyscale logs cluster --id CLUSTER_ID --download --download-dir /tmp
```

### Exploring the Anyscale Metrics Tab

Anyscale provides additional dashboards and metrics that  expand upon the Ray Dashboard to provide more details on your clusters and workloads.

In the **Metrics Tab**, you’ll find general metrics about your Ray cluster. For a more detailed analysis, click **View in Grafana** to open the metrics dashboard directly. From there, you can create custom visualizations to better monitor your applications.

Head over to the Metrics tab to explore some default metrics.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging6.png"  width="500"/>

This includes metrics pertaining to hardware utilization, such as cluster size, memory, and compute usage. To get a lower level understanding, checkout the Ray-speific metrics pertaining to task scheduling, object store memory, and disk spillover. 


<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging7.png"  width="650"/>


If you want to explore the metrics in greater detail, click View in Grafana to open the metrics dashboard directly. These dashboards are organized by each Ray library; in our case, we used only Ray Core.


<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging8.png"  width="400"/>


### Exploring the Ray Dashboard

All Ray Applications, including those runnng on Anyscale, provides a web-based dashboard for monitoring and debugging Ray applications. This tool allows users to track the performance of applications and troubleshoot issues.

Head over to the Dashboard Tab, check out the Jobs section. You can verify the job submitted earlier and explore each one of the 100 tasks we submitted to calculate π.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/logging9.png"  width="600"/>



### Clean Up

When you’re done, remember to terminate your workspace to free up resources and avoid unnecessary costs.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/development12.png" width="700"/>
